In [63]:
%%time
import pandas as pd

CPU times: total: 0 ns
Wall time: 0 ns


In [64]:
%%time
#Check if GPU is successfully detected
import torch

if torch.cuda.is_available():
    print("cuda is available")
else:
    print("cuda is not available")

cuda is available
CPU times: total: 0 ns
Wall time: 0 ns


In [65]:
%%time
df = pd.read_csv('../data/data_cleaned.csv')
df.head()

CPU times: total: 43.2 s
Wall time: 47.7 s


,Unnamed: 0,Id,categories,User_id,review_helpfulness,review_score,review_text,processed_text,tokens
0,0,1882931173,['Comics & Graphic Novels'],AVCGYZL8FQQTD,7/7,4.0,This is only for Julie Strain fans. It's a col...,this is only for julie strain fans its a colle...,"['julie', 'strain', 'fan', 'collection', 'phot..."
1,1,0826414346,['Biography & Autobiography'],A30TK6U7DNS82R,10/10,5.0,I don't care much for Dr. Seuss but after read...,i dont care much for dr seuss but after readin...,"['dont', 'care', 'much', 'dr', 'seuss', 'readi..."
2,2,0826414346,['Biography & Autobiography'],A3UH4UZ4RSVO82,10/11,5.0,"If people become the books they read and if ""t...",if people become the books they read and if th...,"['people', 'become', 'book', 'read', 'child', ..."
3,3,0826414346,['Biography & Autobiography'],A2MVUWT453QH61,7/7,4.0,"Theodore Seuss Geisel (1904-1991), aka &quot;D...",theodore seuss geisel aka quotdr seussquot wa...,"['theodore', 'seuss', 'geisel', 'aka', 'quotdr..."
4,4,0826414346,['Biography & Autobiography'],A22X4XUPKF66MR,3/3,4.0,Philip Nel - Dr. Seuss: American IconThis is b...,philip nel dr seuss american iconthis is basi...,"['philip', 'nel', 'dr', 'seuss', 'american', '..."


In [66]:
df.shape

(2062611, 9)

In [67]:
## We will use only small sets of datasets
#df = df.sample(frac=0.001, random_state=42)
df.shape

(2062611, 9)

In [68]:
#df_sample["tokens"]

In [69]:
%%time
from transformers import BertTokenizer, BertForSequenceClassification
import torch

# Load the pre-trained model and tokenizer
model_name = "bhadresh-savani/bert-base-uncased-emotion"
#model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)

## Move the mode to GPU (if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def get_emotion_bert(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
    inputs = {key: value.to(device) for key, value in inputs.items()}  # Move inputs to GPU
    with torch.no_grad():
        # Get the logits from the model
        outputs = model(**inputs)
        logits = outputs.logits
        # Get the predicted emotion
        emotion = torch.argmax(logits, dim=1).item()
    return emotion

df['review_tokens'] = df['tokens'].apply(lambda tokens: " ".join(tokens))
df['emotions'] = df['review_tokens'].apply(get_emotion_bert)

KeyboardInterrupt: 

In [ ]:
#df["emotions"].value_counts()

In [ ]:
df.head()

In [ ]:
#df_sample.to_csv("emotions.csv")

In [ ]:
#df_sample.columns

In [ ]:
# drop columns not needed
df.drop(columns=['Unnamed: 0', 'processed_text', 'review_tokens'], axis=1, inplace=True)

In [ ]:
df.head()

In [ ]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
#train_data["emotions"]

In [ ]:
test_data.shape

In [ ]:
from sklearn.preprocessing import OneHotEncoder
import pickle

# One-hot encode the emotions for the training and test sets
encoder = OneHotEncoder(sparse_output=False)
train_emotions_ohe = encoder.fit_transform(train_data[['emotions']])
test_emotions_ohe = encoder.transform(test_data[['emotions']])

# Convert the encoded emotions to DataFrames
train_emotions_df = pd.DataFrame(train_emotions_ohe, columns=[f"emotion_{i}" for i in range(train_emotions_ohe.shape[1])])
test_emotions_df = pd.DataFrame(test_emotions_ohe, columns=[f"emotion_{i}" for i in range(test_emotions_ohe.shape[1])])

# Concatenate the encoded emotions with the original data
train_data = pd.concat([train_data.reset_index(drop=True), train_emotions_df], axis=1)
test_data = pd.concat([test_data.reset_index(drop=True), test_emotions_df], axis=1)

# Drop the 'emotions' column if not needed
train_data.drop('emotions', axis=1, inplace=True)
test_data.drop('emotions', axis=1, inplace=True)

# Display the size of the splits and the first few rows
print(f"Training data size: {train_data.shape}")
print(f"Test data size: {test_data.shape}")
print(train_data.head())
print(test_data.head())

# Save encoder to a pkl file
with open("../models/emotion_encoder.pkl", "wb") as f:
    pickle.dump(encoder, f)

print("Encoder saved successfully.")

In [ ]:
train_data.head()

In [ ]:
#train_emotions_df

In [ ]:
train_data.head()

In [ ]:

train_data.to_csv('../data/train_data.csv', index=False)
test_data.to_csv('../data/test_data.csv', index=False)